## Deliverable 2. Create a Customer Travel Destinations Map.

In [4]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key2

# Configure gmaps API key
gmaps.configure(api_key=g_key2)

In [11]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Jurado,CO,7.1053,-77.7641,82.72,83,100,5.50,moderate rain
1,1,Bluff,NZ,-46.6000,168.3333,49.80,89,100,8.32,light rain
2,2,Balkhash,KZ,46.8440,74.9804,74.41,46,63,21.09,broken clouds
3,3,Mataura,NZ,-46.1927,168.8643,49.48,99,100,1.16,light rain
4,4,Butaritari,KI,3.0707,172.7902,83.52,78,77,20.69,light rain


In [12]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [14]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Jurado,CO,7.1053,-77.7641,82.72,83,100,5.50,moderate rain
2,2,Balkhash,KZ,46.8440,74.9804,74.41,46,63,21.09,broken clouds
4,4,Butaritari,KI,3.0707,172.7902,83.52,78,77,20.69,light rain
8,8,Tigre,AR,-34.4260,-58.5796,72.23,53,9,4.00,clear sky
9,9,Komsomolskiy,UZ,40.4272,71.7189,74.62,41,17,3.24,few clouds
11,11,Gat,IL,31.6100,34.7642,79.39,67,0,3.89,clear sky
15,15,Bani Walid,LY,31.7566,13.9942,82.63,24,98,13.98,overcast clouds
17,17,Hilo,US,19.7297,-155.0900,80.80,69,40,0.00,scattered clouds
18,18,Souillac,MU,-20.5167,57.5167,72.36,86,91,7.49,overcast clouds
19,19,Kapaa,US,22.0752,-159.3190,78.78,80,90,1.01,light rain


In [16]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [18]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df
clean_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Jurado,CO,7.1053,-77.7641,82.72,83,100,5.50,moderate rain
2,2,Balkhash,KZ,46.8440,74.9804,74.41,46,63,21.09,broken clouds
4,4,Butaritari,KI,3.0707,172.7902,83.52,78,77,20.69,light rain
8,8,Tigre,AR,-34.4260,-58.5796,72.23,53,9,4.00,clear sky
9,9,Komsomolskiy,UZ,40.4272,71.7189,74.62,41,17,3.24,few clouds
11,11,Gat,IL,31.6100,34.7642,79.39,67,0,3.89,clear sky
15,15,Bani Walid,LY,31.7566,13.9942,82.63,24,98,13.98,overcast clouds
17,17,Hilo,US,19.7297,-155.0900,80.80,69,40,0.00,scattered clouds
18,18,Souillac,MU,-20.5167,57.5167,72.36,86,91,7.49,overcast clouds
19,19,Kapaa,US,22.0752,-159.3190,78.78,80,90,1.01,light rain


In [19]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Jurado,CO,82.72,moderate rain,7.1053,-77.7641,
2,Balkhash,KZ,74.41,broken clouds,46.8440,74.9804,
4,Butaritari,KI,83.52,light rain,3.0707,172.7902,
8,Tigre,AR,72.23,clear sky,-34.4260,-58.5796,
9,Komsomolskiy,UZ,74.62,few clouds,40.4272,71.7189,
11,Gat,IL,79.39,clear sky,31.6100,34.7642,
15,Bani Walid,LY,82.63,overcast clouds,31.7566,13.9942,
17,Hilo,US,80.80,scattered clouds,19.7297,-155.0900,
18,Souillac,MU,72.36,overcast clouds,-20.5167,57.5167,
19,Kapaa,US,78.78,light rain,22.0752,-159.3190,


In [32]:
# 6a. Set parameters to search for hotels with 5000 meters.

params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key2
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
     # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url,params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(KeyError, IndexError):
        print("Hotel not found ....skipping.")
        
hotel_df.head(10)        

Hotel not found ....skipping.
Hotel not found ....skipping.
Hotel not found ....skipping.
Hotel not found ....skipping.
Hotel not found ....skipping.
Hotel not found ....skipping.
Hotel not found ....skipping.
Hotel not found ....skipping.
Hotel not found ....skipping.
Hotel not found ....skipping.
Hotel not found ....skipping.
Hotel not found ....skipping.
Hotel not found ....skipping.
Hotel not found ....skipping.
Hotel not found ....skipping.
Hotel not found ....skipping.
Hotel not found ....skipping.
Hotel not found ....skipping.
Hotel not found ....skipping.
Hotel not found ....skipping.
Hotel not found ....skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Jurado,CO,82.72,moderate rain,7.1053,-77.7641,
2,Balkhash,KZ,74.41,broken clouds,46.8440,74.9804,Family
4,Butaritari,KI,83.52,light rain,3.0707,172.7902,Isles Sunset Lodge
8,Tigre,AR,72.23,clear sky,-34.4260,-58.5796,Puerto Delta Apartamentos
9,Komsomolskiy,UZ,74.62,few clouds,40.4272,71.7189,Ikathouse
11,Gat,IL,79.39,clear sky,31.6100,34.7642,OlusHome
15,Bani Walid,LY,82.63,overcast clouds,31.7566,13.9942,منزل الصابر أبوفانة
17,Hilo,US,80.80,scattered clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
18,Souillac,MU,72.36,overcast clouds,-20.5167,57.5167,Shanti Maurice Resort & Spa
19,Kapaa,US,78.78,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort


In [40]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"]!= ""]
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Balkhash,KZ,74.41,broken clouds,46.8440,74.9804,Family
4,Butaritari,KI,83.52,light rain,3.0707,172.7902,Isles Sunset Lodge
8,Tigre,AR,72.23,clear sky,-34.4260,-58.5796,Puerto Delta Apartamentos
9,Komsomolskiy,UZ,74.62,few clouds,40.4272,71.7189,Ikathouse
11,Gat,IL,79.39,clear sky,31.6100,34.7642,OlusHome
15,Bani Walid,LY,82.63,overcast clouds,31.7566,13.9942,منزل الصابر أبوفانة
17,Hilo,US,80.80,scattered clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
18,Souillac,MU,72.36,overcast clouds,-20.5167,57.5167,Shanti Maurice Resort & Spa
19,Kapaa,US,78.78,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
22,Atuona,PF,78.93,few clouds,-9.8000,-139.0333,Villa Enata


In [42]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [45]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [48]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

### End of Deliverable 2.